### Prompt:
You are an analyst for a fastfood restaurant. The company has tasked you to analyze their past transactions. Each row represents a single transaction/order from one of their ten (10) branches. The order is recorded as a __`string`__ which is formatted as: 

"`[quantity1][item_code1], [quantity2][item_code2], [quantity3][item_code3]`"

However, because of some technical error, the __number of combo meals purchased__ as well as the __total order value__ was lost, so we'll need to restore that.

### You are tasked to come up with the following:
1. Given the order column, count how many of each product was purchased
2. Given each product count, get the number of supposed combo meals purchased. <br><i>(You may update the product count when the combo meals are counted)</i>.
3. Calculate the total order value.
4. Determine which branch made the most revenue, and how much?

### The combo meals are detailed below:
1. Family combo: 4 burgers, 4 fries, 4 drinks
2. Big combo: 1 burgers, 1 fries, 1 drinks
3. Snack combo: 1 fries, 1 drinks

In [35]:
import pandas as pd
import numpy as np

df = pd.read_csv("food_transactions.csv")
df

,Unnamed: 0,date,branch,order
0,0,2023-06-26 21:36:39,Branch 3,"1S_Burger, 4S_Fries, 5S_Drink"
1,1,2023-12-22 10:57:56,Branch 9,"2S_Burger, 4S_Fries, 3S_Drink"
2,2,2023-12-14 14:07:51,Branch 8,"2S_Burger, 4S_Fries, 2S_Drink"
3,3,2023-11-01 02:10:22,Branch 4,"6S_Fries, 1S_Drink"
4,4,2023-09-30 07:42:12,Branch 3,"5S_Burger, 6S_Drink"
...,...,...,...,...
134770,134770,2023-12-11 04:21:04,Branch 8,"3S_Fries, 6S_Drink"
134771,134771,2023-08-25 18:43:05,Branch 8,"7S_Burger, 6S_Fries"
134772,134772,2023-09-24 14:15:58,Branch 10,"4S_Burger, 6S_Fries"
134773,134773,2023-07-08 12:43:10,Branch 4,"7S_Burger, 6S_Fries, 5S_Drink"


In [3]:
jsondf = pd.read_json("price_list.json")
jsondf

,price
Big_combo,169
Fam_combo,599
S_Burger,89
S_Drink,49
S_Fries,69
Snack_combo,99


In [83]:
# 1. Given the order column, count how many of each product was purchased

df = pd.read_csv("food_transactions.csv")
df['list products'] = df['order'].apply(lambda x: x.split(', ') if type(x) == str else x)
df['order'] = df['order'].apply(lambda x: str(x)) 

# df[df['order'].str.contains('S_Burger')]

# df[df['order']

df['Burger True'] = df.apply(lambda x: 'S_Burger' in x['order'], axis=1)
df['Remove Burger'] = df.apply(lambda x: x['list products'][1:] if x['Burger True']==True else x['list products'], axis=1)
df['Burger Count'] = df.apply(lambda x: x['list products'][0][0] if x['Burger True'] else 0, axis=1)

df['Fries True'] = df.apply(lambda x: 'S_Fries' in x['order'], axis=1)
df['Remove Fries'] = df.apply(lambda x: x['Remove Burger'][1:] if x['Fries True']==True else x['list products'], axis=1)
df['Fries Count'] = df.apply(lambda x: x['Remove Burger'][0][0] if x['Fries True'] else 0, axis=1)

df['Drinks True'] = df.apply(lambda x: 'S_Drink' in x['order'], axis=1)
df['Remove Drinks'] = df.apply(lambda x: x['Remove Fries'][1:] if x['Drinks True']==True else x['list products'], axis=1)
df['Drinks Count'] = df.apply(lambda x: x['Remove Fries'][0][0] if x['Drinks True'] else 0, axis=1)

# df['Burger Count'].sum()

df

,Unnamed: 0,date,branch,order,list products,Burger True,Remove Burger,Burger Count,Fries True,Remove Fries,Fries Count,Drinks True,Remove Drinks,Drinks Count
0,0,2023-06-26 21:36:39,Branch 3,"1S_Burger, 4S_Fries, 5S_Drink","[1S_Burger, 4S_Fries, 5S_Drink]",True,"[4S_Fries, 5S_Drink]",1,True,[5S_Drink],4,True,[],5
1,1,2023-12-22 10:57:56,Branch 9,"2S_Burger, 4S_Fries, 3S_Drink","[2S_Burger, 4S_Fries, 3S_Drink]",True,"[4S_Fries, 3S_Drink]",2,True,[3S_Drink],4,True,[],3
2,2,2023-12-14 14:07:51,Branch 8,"2S_Burger, 4S_Fries, 2S_Drink","[2S_Burger, 4S_Fries, 2S_Drink]",True,"[4S_Fries, 2S_Drink]",2,True,[2S_Drink],4,True,[],2
3,3,2023-11-01 02:10:22,Branch 4,"6S_Fries, 1S_Drink","[6S_Fries, 1S_Drink]",False,"[6S_Fries, 1S_Drink]",0,True,[1S_Drink],6,True,[],1
4,4,2023-09-30 07:42:12,Branch 3,"5S_Burger, 6S_Drink","[5S_Burger, 6S_Drink]",True,[6S_Drink],5,False,"[5S_Burger, 6S_Drink]",0,True,[6S_Drink],5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134770,134770,2023-12-11 04:21:04,Branch 8,"3S_Fries, 6S_Drink","[3S_Fries, 6S_Drink]",False,"[3S_Fries, 6S_Drink]",0,True,[6S_Drink],3,True,[],6
134771,134771,2023-08-25 18:43:05,Branch 8,"7S_Burger, 6S_Fries","[7S_Burger, 6S_Fries]",True,[6S_Fries],7,True,[],6,False,"[7S_Burger, 6S_Fries]",0
134772,134772,2023-09-24 14:15:58,Branch 10,"4S_Burger, 6S_Fries","[4S_Burger, 6S_Fries]",True,[6S_Fries],4,True,[],6,False,"[4S_Burger, 6S_Fries]",0
134773,134773,2023-07-08 12:43:10,Branch 4,"7S_Burger, 6S_Fries, 5S_Drink","[7S_Burger, 6S_Fries, 5S_Drink]",True,"[6S_Fries, 5S_Drink]",7,True,[5S_Drink],6,True,[],5


In [122]:
# 2. Given each product count, get the number of supposed combo meals purchased.
# (You may update the product count when the combo meals are counted).
# The combo meals are detailed below:
# Family combo: 4 burgers, 4 fries, 4 drinks
# Big combo: 1 burgers, 1 fries, 1 drinks
# Snack combo: 1 fries, 1 drinks


# one_combo = df[(df['order'] == '4S_Burger, 4S_Fries, 4S_Drink') | (df['order'] == '1S_Burger, 1S_Fries, 1S_Drink') | (df['order'] == '1S_Fries, 1S_Drink')].count()['order']

df['Burger Count'] = df['Burger Count'].apply(lambda x: int(x))
df['Fries Count'] = df['Fries Count'].apply(lambda x: int(x))
df['Drinks Count'] = df['Drinks Count'].apply(lambda x: int(x))

df

# family_combo = df[(df['Burger Count'] % 4 == 0) & (df['Fries Count'] % 4 == 0) & (df['Drinks Count'] % 4 == 0)].count()['order']
# family_combo

# big_combo = df[(df['Burger Count'] % 4 != 0) & (df['Fries Count'] % 4 != 0) & (df['Drinks Count'] % 4 != 0) & (df['Burger Count'] == df['Fries Count']) & (df['Fries Count'] == df['Drinks Count'])].count()['order']

# big_combo_df = df[(df['Burger Count'] % 4 != 0) & (df['Fries Count'] % 4 != 0) & (df['Drinks Count'] % 4 != 0) & (df['Burger Count'] == df['Fries Count']) & (df['Fries Count'] == df['Drinks Count'])]

df['big combo count'] = df.apply(lambda x: x['Burger Count'] % 4 if x['Burger Count'] == x['Fries Count'] and x['Burger Count'] == x['Drinks Count'] else 0, axis=1)
df['family combo count'] = df.apply(lambda x: x['Burger Count'] // 4 if x['Burger Count'] == x['Fries Count'] and x['Burger Count'] == x['Drinks Count'] else 0, axis=1)
df['snack combo'] = df.apply(lambda x: x['Fries Count'] if x['Burger Count'] == 0 and x['Drinks Count'] == x['Fries Count'] and x['big combo count'] == 0 else 0, axis=1)

df['snack combo'].sum() + df['big combo count'].sum() + df['family combo count'].sum()

11754

In [123]:
# 3. Calculate the total order value.

jsondf


,price
Big_combo,169
Fam_combo,599
S_Burger,89
S_Drink,49
S_Fries,69
Snack_combo,99


In [149]:
prices = jsondf.to_dict()

# del df['prices']
df['fam combo price'] = df['family combo count'].apply(lambda x: prices['price']['Fam_combo']*x if x > 0 else 0)
df['big combo price'] = df['big combo count'].apply(lambda x: prices['price']['Big_combo']*x if x > 0 else 0)
df['snack combo price'] = df['snack combo'].apply(lambda x: prices['price']['Snack_combo']*x if x > 0 else 0)

big_combo_df = df[(df['Burger Count'] % 4 != 0) & (df['Fries Count'] % 4 != 0) & (df['Drinks Count'] % 4 != 0) & (df['Burger Count'] == df['Fries Count']) & (df['Fries Count'] == df['Drinks Count'])]
snack_combo_df = df[(df['Burger Count'] == 0) & (df['Drinks Count'] == df['Fries Count']) & (df['big combo count'] == 0)]

df['remaining burger'] = df.apply(lambda x: x['Burger Count'] - (x['family combo count']*4 + x['big combo count'] + x['snack combo']), axis=1)
df['remaining fries'] = df.apply(lambda x: x['Fries Count'] - (x['family combo count']*4 + x['big combo count'] + x['snack combo']), axis=1)
df['remaining drinks'] = df.apply(lambda x: x['Drinks Count'] - (x['family combo count']*4 + x['big combo count'] + x['snack combo']), axis=1)

df['burger price'] = df['remaining burger'].apply(lambda x: x*prices['price']['S_Burger'])
df['fries price'] = df['remaining fries'].apply(lambda x: x*prices['price']['S_Fries'])
df['drinks price'] = df['remaining drinks'].apply(lambda x: x*prices['price']['S_Drink'])

df['total order value'] = df['fam combo price'] + df['big combo price'] + df['snack combo price'] + df['burger price'] + df['fries price'] + df['drinks price']

# list(df.columns)


,Unnamed: 0,date,branch,order,list products,Burger True,Remove Burger,Burger Count,Fries True,Remove Fries,...,fam combo price,big combo price,snack combo price,remaining burger,remaining fries,remaining drinks,burger price,fries price,drinks price,total order value
0,0,2023-06-26 21:36:39,Branch 3,"1S_Burger, 4S_Fries, 5S_Drink","[1S_Burger, 4S_Fries, 5S_Drink]",True,"[4S_Fries, 5S_Drink]",1,True,[5S_Drink],...,0,0,0,1,4,5,89,276,245,610
1,1,2023-12-22 10:57:56,Branch 9,"2S_Burger, 4S_Fries, 3S_Drink","[2S_Burger, 4S_Fries, 3S_Drink]",True,"[4S_Fries, 3S_Drink]",2,True,[3S_Drink],...,0,0,0,2,4,3,178,276,147,601
2,2,2023-12-14 14:07:51,Branch 8,"2S_Burger, 4S_Fries, 2S_Drink","[2S_Burger, 4S_Fries, 2S_Drink]",True,"[4S_Fries, 2S_Drink]",2,True,[2S_Drink],...,0,0,0,2,4,2,178,276,98,552
3,3,2023-11-01 02:10:22,Branch 4,"6S_Fries, 1S_Drink","[6S_Fries, 1S_Drink]",False,"[6S_Fries, 1S_Drink]",0,True,[1S_Drink],...,0,0,0,0,6,1,0,414,49,463
4,4,2023-09-30 07:42:12,Branch 3,"5S_Burger, 6S_Drink","[5S_Burger, 6S_Drink]",True,[6S_Drink],5,False,"[5S_Burger, 6S_Drink]",...,0,0,0,5,0,5,445,0,245,690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134770,134770,2023-12-11 04:21:04,Branch 8,"3S_Fries, 6S_Drink","[3S_Fries, 6S_Drink]",False,"[3S_Fries, 6S_Drink]",0,True,[6S_Drink],...,0,0,0,0,3,6,0,207,294,501
134771,134771,2023-08-25 18:43:05,Branch 8,"7S_Burger, 6S_Fries","[7S_Burger, 6S_Fries]",True,[6S_Fries],7,True,[],...,0,0,0,7,6,0,623,414,0,1037
134772,134772,2023-09-24 14:15:58,Branch 10,"4S_Burger, 6S_Fries","[4S_Burger, 6S_Fries]",True,[6S_Fries],4,True,[],...,0,0,0,4,6,0,356,414,0,770
134773,134773,2023-07-08 12:43:10,Branch 4,"7S_Burger, 6S_Fries, 5S_Drink","[7S_Burger, 6S_Fries, 5S_Drink]",True,"[6S_Fries, 5S_Drink]",7,True,[5S_Drink],...,0,0,0,7,6,5,623,414,245,1282


In [150]:
# 4. Determine which branch made the most revenue, and how much?


final_df = df[['branch','total order value']]
final_df.groupby('branch').sum(numeric_only=True)['total order value'].sort_values()

branch
Branch 5     9503160
Branch 2     9554953
Branch 8     9560807
Branch 7     9605117
Branch 10    9625376
Branch 3     9658002
Branch 6     9658885
Branch 1     9671232
Branch 9     9695102
Branch 4     9738883
Name: total order value, dtype: int64